In [1]:
from ibapi.client import *
from ibapi.wrapper import *
from ibapi.contract import Contract  # For creating stock contracts
from ibapi.order import Order        # For constructing orders

import threading
import time

In [2]:
class TestApp(EClient, EWrapper):
    
    def __init__(self):
        """
        Initialize the TestApp as both a client and a wrapper for IB API.
        """
        EClient.__init__(self, self)
        # This variable will hold the next valid order ID provided by IB.
        self.nextOrderId = None

    
    def nextValidId(self, orderId: int):
        """
        Callback method called when the IB API provides a valid order ID.
        We simply store it in an instance variable.
        """
        self.nextOrderId = orderId
        print(f"Next valid order ID stored: {orderId}")

    
    def createBracketOrder(self, symbol: str, quantity: int, parentPrice: float,
                           profitPrice: float, stopPrice: float):
        """
        Creates and submits a bracket order composed of three linked orders:
        
          1. Parent Order:
             - A BUY limit order that opens the position at a specific price.
          2. Profit Taker:
             - A SELL limit order placed at a target price (profitPrice)
               to take profits when the market moves in your favor.
          3. Stop Loss:
             - A SELL stop order with a trigger price (stopPrice) that helps
               limit losses if the market moves against you.
        
        This method must be called externally after the app has connected
        and a valid order ID is received.

        Parameters:
          symbol      : The ticker symbol for the asset (e.g., "AAPL").
          quantity    : The number of shares to trade.
          parentPrice : The limit entry price for the BUY order.
          profitPrice : The limit price at which the profit taker order executes.
          stopPrice   : The trigger price for the stop loss order.
        """
        # Ensure that a valid order ID has been received.
        if self.nextOrderId is None:
            print("Error: nextOrderId is not set. Connect to IB and wait for nextValidId callback.")
            return

        # ------------------------------
        # Define the contract for the underlying asset.
        # ------------------------------
        contract = Contract()
        contract.symbol = symbol             # e.g., "AAPL"
        contract.secType = "STK"               # Stock security type
        contract.exchange = "SMART"            # Use IB's SMART routing
        contract.currency = "USD"              # Currency in USD

        # ------------------------------
        # Create the Parent Order (Entry Order).
        # ------------------------------
        parent = Order()
        parent.orderId = self.nextOrderId            # Use the stored valid order ID
        parent.action = "BUY"                        # BUY to open a long position
        parent.orderType = "LMT"                     # Use a limit order for precision
        parent.lmtPrice = parentPrice                # User-supplied entry price
        parent.totalQuantity = quantity              # Number of shares to buy
        parent.transmit = False                      # Hold transmission until the bracket is complete

        # ------------------------------
        # Create the Profit Taker Order (Take-Profit Order).
        # ------------------------------
        profitTaker = Order()
        profitTaker.orderId = self.nextOrderId + 1     # Next sequential order ID
        profitTaker.parentId = parent.orderId          # Link this order to the parent
        profitTaker.action = "SELL"                    # SELL order to exit the long position
        profitTaker.orderType = "LMT"                  # Limit order ensures execution at the desired price
        profitTaker.lmtPrice = profitPrice             # Target price for profit taking (above parentPrice for long)
        profitTaker.totalQuantity = quantity           # Must match the parent's quantity
        profitTaker.transmit = False                   # Hold transmission until final order is added

        # ------------------------------
        # Create the Stop Loss Order (Risk Management Order).
        # ------------------------------
        stopLoss = Order()
        stopLoss.orderId = self.nextOrderId + 2         # Next sequential order ID
        stopLoss.parentId = parent.orderId              # Link this order to the parent
        stopLoss.action = "SELL"                        # SELL order for exiting the long if adverse movement occurs
        stopLoss.orderType = "STP"                      # Stop order triggers a market order when the stop price is hit
        stopLoss.auxPrice = stopPrice                   # The stop price that triggers the order (below parentPrice for long)
        stopLoss.totalQuantity = quantity               # Must match the parent's quantity
        # For bracket orders, the final leg (stop loss here) carries transmit=True,
        # causing the entire bracket (all orders) to be sent together.
        stopLoss.transmit = True

        # ------------------------------
        # Submit the bracket orders.
        # ------------------------------
        print(f"Placing Parent Order (ID {parent.orderId}): BUY {quantity} {symbol} @ {parentPrice}")
        self.placeOrder(parent.orderId, contract, parent)
        
        print(f"Placing Profit Taker Order (ID {profitTaker.orderId}): SELL {quantity} {symbol} @ {profitPrice}")
        self.placeOrder(profitTaker.orderId, contract, profitTaker)
        
        print(f"Placing Stop Loss Order (ID {stopLoss.orderId}): SELL {quantity} {symbol} if price falls to {stopPrice}")
        self.placeOrder(stopLoss.orderId, contract, stopLoss)

    
    def openOrder(self, orderId: OrderId, contract: Contract, order: Order, orderState: OrderState):
        """
        Callback method that prints out order details when an order is opened or updated.
        """
        print(f"openOrder: OrderID: {orderId}, Contract: {contract}, Order: {order}, State: {orderState}")

    
    def orderStatus(self, orderId: OrderId, status: str, filled: float, remaining: float,
                    avgFillPrice: float, permId: int, parentId: int, lastFillPrice: float,
                    clientId: int, whyHeld: str, mktCapPrice: float):
        """
        Callback method providing real-time updates on the status of orders.
        """
        print(f"orderStatus: OrderID: {orderId}, Status: {status}, Filled: {filled}, Remaining: {remaining}, "
              f"AvgFillPrice: {avgFillPrice}, ParentID: {parentId}")

    
    def execDetails(self, reqId: int, contract: Contract, execution: Execution):
        """
        Callback method that provides detailed information on trade executions.
        """
        print(f"execDetails: ReqID: {reqId}, Contract: {contract}, Execution: {execution}")



In [3]:
port = 7496  # Typical port for connecting to TWS (7496 for IB Gateway live trading)
clientId = 7

# Create an instance of the TestApp and connect to TWS.
app = TestApp()
app.connect("127.0.0.1", port, clientId)

# Start the API processing loop in a separate thread so that it does not block the main thread.
threading.Thread(target=app.run).start()
time.sleep(1)  # Pause briefly to ensure a reliable connection before making requests

ERROR -1 1746602798580 2104 Market data farm connection is OK:cashfarm
ERROR -1 1746602798581 2104 Market data farm connection is OK:usfarm.nj
ERROR -1 1746602798581 2104 Market data farm connection is OK:eufarm
ERROR -1 1746602798581 2104 Market data farm connection is OK:usfarm
ERROR -1 1746602798582 2106 HMDS data farm connection is OK:euhmds
ERROR -1 1746602798582 2106 HMDS data farm connection is OK:fundfarm
ERROR -1 1746602798582 2106 HMDS data farm connection is OK:ushmds
ERROR -1 1746602798582 2158 Sec-def data farm connection is OK:secdefeu


Next valid order ID stored: 20


In [4]:
# Now that the order id is stored, we can externally call createBracketOrder.
# For a long bracket order, e.g.:
#  - Entry (parent) BUY at 140, 
#  - Profit Taker SELL at 150 (above entry), 
#  - Stop Loss SELL if price falls to 137 (below entry).
app.createBracketOrder(symbol="NVDA", quantity=1, parentPrice=113, profitPrice=150, stopPrice=100)

# # Keep the script running for demonstration purposes.
# time.sleep(10)

Placing Parent Order (ID 20): BUY 1 NVDA @ 113
Placing Profit Taker Order (ID 21): SELL 1 NVDA @ 150
Placing Stop Loss Order (ID 22): SELL 1 NVDA if price falls to 100
